In [2]:
import numpy as np 
import torch 


/export/home/ffeiden/miniconda3/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
channel_mult = [1, 2, 4, 4]
num_res_blocks = 2
num_res_blocks = len(channel_mult) * [num_res_blocks]
print(num_res_blocks)

[2, 2, 2, 2]


In [8]:
model_channels = 320
num_res_blocks = [2, 2, 2, 2]
attention_resolution = [ 4, 2, 1 ]
num_head_channels = 64
context_dim = 1_024

input_blocks = ['Conv2D: in: 4, out: 320, kernel: 3, padding: 1']

feature_size = model_channels
input_block_chans = [model_channels]
ch = model_channels
ds = 1
for level, mult in enumerate(channel_mult):
    for nr in range(num_res_blocks[level]):
        layers =[ f'ResBlock: Chan: {ch}, time_embd_dim: 1280; dropout: 0; out_chan: {mult * model_channels}; scale_shift_norm: False']
        
        ch = mult * model_channels
        if ds in attention_resolution: 
            num_heads = ch // num_head_channels
            dim_head = num_head_channels

        layers.append( (f'AttentionBlock: Chan: {ch}; n_head: {num_heads}; n_head_ch: {num_head_channels}'
                        ) if not True else (f'SpacialTransformer: ch: {ch}; n_head: {num_heads}; n_head_ch: {num_head_channels}, depth: 1, context_dim: {context_dim}, disable_selfAtt: False, use_linear: True'))
        
        input_blocks.append(layers)
        feature_size += ch
        input_block_chans.append(ch)

        if level != len(channel_mult) -1 : 
            out_ch = ch 
            input_blocks.append(f'ResBlock: Chan: {ch}, time_embd_dim: 1280; dropout: 0; out_chan: {out_ch}; scale_shift_norm: False; down: True' if False 
                                else f'Downsample: Chans: {ch}; conv_resample: True; out_chans: {out_ch}')
        
            ch = out_ch
            input_block_chans.append(ch)
            ds *=2
            feature_size += ch

for str in input_blocks: 
    print(str)

Conv2D: in: 4, out: 320, kernel: 3, padding: 1
['ResBlock: Chan: 320, time_embd_dim: 1280; dropout: 0; out_chan: 320; scale_shift_norm: False', 'SpacialTransformer: ch: 320; n_head: 5; n_head_ch: 64, depth: 1, context_dim: 1024, disable_selfAtt: False, use_linear: True']
Downsample: Chans: 320; conv_resample: True; out_chans: 320
['ResBlock: Chan: 320, time_embd_dim: 1280; dropout: 0; out_chan: 320; scale_shift_norm: False', 'SpacialTransformer: ch: 320; n_head: 5; n_head_ch: 64, depth: 1, context_dim: 1024, disable_selfAtt: False, use_linear: True']
Downsample: Chans: 320; conv_resample: True; out_chans: 320
['ResBlock: Chan: 320, time_embd_dim: 1280; dropout: 0; out_chan: 640; scale_shift_norm: False', 'SpacialTransformer: ch: 640; n_head: 10; n_head_ch: 64, depth: 1, context_dim: 1024, disable_selfAtt: False, use_linear: True']
Downsample: Chans: 640; conv_resample: True; out_chans: 640
['ResBlock: Chan: 640, time_embd_dim: 1280; dropout: 0; out_chan: 640; scale_shift_norm: False', 